In [3]:
%pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 52.4 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## create dummy data

In [1]:
import pandas as pd
tables=pd.read_html('https://en.wikipedia.org/wiki/ISO_3166-1')

In [2]:
tables[0].iloc[0:1]

,Short name upper case in ISO 3166,Short name lower case in ISO 3166 Full name,Listed as independent in ISO 3166,Local short name,Dispute,Link to ISO 3166-2
0,AFGHANISTAN,Afghanistanthe Islamic Republic of Afghanistan,Yes[6],Afghānistān (ps) Afghānistān (fa),Under the unrecognized government of the Islam...,ISO 3166-2:AF


In [3]:
tables[0].to_csv('dir_diff/dir1/disputed.csv',index=False) 
tables[0].to_csv('dir_diff/dir2/disputed.csv',index=False) 

In [4]:
tables[1].iloc[0:1]

,English short name (using title case),Alpha-2 code,Alpha-3 code,Numeric code,Link to ISO 3166-2,Independent
0,Afghanistan[b],AF,AFG,4,ISO 3166-2:AF,Yes


In [5]:
tables[1].to_csv('dir_diff/dir1/name_code.csv',index=False) 
tables[1].to_csv('dir_diff/dir2/name_code.csv',index=False) 
tables[1].to_csv('dir_diff/dir2/~name_code.csv',index=False) #additional copy

In [6]:
tables[2].iloc[0:1]

,Edition/Newsletter,Date issued,Contents
0,ISO 3166:1974,1974,First edition of ISO 3166


In [7]:
tables[2].to_csv('dir_diff/dir1/version_control.csv',index=False) 
tables[2].iloc[2:].to_csv('dir_diff/dir2/version_control.csv',index=False) #corrupted copy

## Directory comparison

In [12]:
!ls -l dir_diff/*/* > diff.txt
list_of_headers=['permissions', 'hard_links','user','group','size','month','day','time','path']
dir_listing=pd.read_csv('diff.txt', sep='\s+',names=list_of_headers)

In [13]:
def split_path(df):
    return (df
            .assign(parent_dir=df
                    .path
                    .str.split(pat='/',expand=True)[0],
                    inner_dir=df
                    .path
                    .str.split(pat='/',expand=True)[1],
                    filename=df
                    .path
                    .str.split(pat='/',expand=True)[2]
                    )
            )

In [14]:
dir_listing.pipe(split_path).groupby(['inner_dir']).agg({'filename':'count','size':'sum'})

,filename,size
inner_dir,,
dir1,3,16686
dir2,4,26986


In [15]:
(dir_listing
 .pipe(split_path)
 .groupby(['filename','inner_dir'])
 .agg({'filename':'count','size':'sum'}) # Identify additional file in dir2, inconsistent size
 .unstack()
)

filename          size         
inner_dir               dir1 dir2     dir1     dir2
filename                                           
disputed.csv             1.0  1.0   1120.0   1120.0
name_code.csv            1.0  1.0  10391.0  10391.0
version_control.csv      1.0  1.0   5175.0   5084.0
~name_code.csv           NaN  1.0      NaN  10391.0